In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
from pprint import pprint

import csv

mmCourse = 'lgdm'

sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()

mmCoursesMeta = {
    'buslead': {
        "buslead1x": {"title": "UQx_BUSLEAD1x",  'grade': 0.65},
        "buslead2x": {"title": "UQx_BUSLEAD2x",  'grade': 0.65},
        "buslead3x": {"title": "UQx_BUSLEAD3x",  'grade': 0.65},
        "buslead4x": {"title": "UQx_BUSLEAD4x",  'grade': 0.65},
        "buslead5x": {"title": "UQx_BUSLEAD5x",  'grade': 0.65},
    },
    'corpinn': {
        "corpinn1x": {"title": "UQx_CORPINN1x", 'grade': 0.7},
        "corpinn2x": {"title": "UQx_CORPINN2x",  'grade': 0.7},
        "corpinn3x": {"title": "UQx_CORPINN3x",  'grade': 0.7},
        "corpinn4x": {"title": "UQx_CORPINN4x",  'grade': 0.7},
        "corpinn5x": {"title": "UQx_CORPINN5x",  'grade': 0.5},
    },
    'engy': {
        "engy0x": {"title": "UQx_ENGY0x", 'grade': 0.7},
        "engy1x": {"title": "UQx_ENGY1x", 'grade': 0.7},
        "engy2x": {"title": "UQx_ENGY2x", 'grade': 0.7},
        "engy3x": {"title": "UQx_ENGY3x", 'grade': 0.7},
        "engycapx": {"title": "UQx_ENGYCAPx", 'grade': 0.7},
    },
    'lgdm': {
        "lgdm1x": {"title": "UQx_LGDM1x", 'grade': 0.65},
        "lgdm2x": {"title": "UQx_LGDM2x",  'grade': 0.65},
        "lgdm3x": {"title": "UQx_LGDM3x",  'grade': 0.65},
        "lgdm4x": {"title": "UQx_LGDM4x",  'grade': 0.65},
        "lgdm5x": {"title": "UQx_LGDM5x",  'grade': 0.65},
    },
}
mmCoursesSeries = {
    'buslead': ['buslead1x', 'buslead2x', 'buslead3x', 'buslead4x'],
    'corpinn': ['corpinn1x', 'corpinn2x', 'corpinn3x', 'corpinn4x'],
    'engy': ['engy0x', 'engy1x', 'engy2x', 'engy3x'],
    'lgdm': ['lgdm1x', 'lgdm2x', 'lgdm3x', 'lgdm4x'],
}

micromasters_courses = mmCoursesMeta[mmCourse]
micromasters_course_series = mmCoursesSeries[mmCourse]

def update_course_instancedetails_auditpassed():
    print('update_course_instancedetails_auditpassed')
    ci_query = 'SELECT * FROM course_instancedetails'
    result = sqlite_connection.execute(ci_query)
    for row in result:
        update_id = int(row[0])
        course = row[6]
        pass_line = micromasters_courses[course]['grade']
        #print('pass_line', pass_line)
        grade = row[2]
        enrollment_track = row[3]
        update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('N', update_id)
        if (enrollment_track == 'audit') and (grade >= pass_line):
            update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('Y', update_id)
        sqlite_connection.execute(update_ci)
        
        
def micromasters_courses_add_last_run():
    for course in micromasters_courses:
        query = "SELECT course_instance FROM course_instancedetails WHERE course='%s' ORDER BY year DESC, term DESC;" % (course)
        query_result = sqlite_connection.execute(query).fetchone()
        if query_result is not None:
            micromasters_courses[course]['last_run'] = query_result[0]
    #pprint(micromasters_courses)
    

def find_audit_passed(course, pointed_run=''):
    email_list = []
    if pointed_run == '':
        pointed_run = micromasters_courses[course]['last_run']
    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.audit_passed = 'Y'" % (pointed_run)
    #print(query)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        email_list.append(record['email'])
    pointed_course_instances[course]['auditpassed_emails'] = email_list
    
def find_certificate_eligible(course, pointed_run=''):
    email_list = []
    if pointed_run == '':
        pointed_run = micromasters_courses[course]['last_run']
    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.certificate_eligible = 'Y'" % (pointed_run)
    #print(query)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        email_list.append(record['email'])
    pointed_course_instances[course]['certificate_eligible_emails'] = email_list
        

# Only four courses should be included        
pointed_course_instances = {
    'lgdm1x': {
        'current_run':'UQx_LGDM1x_1T2017',
        'auditpassed_emails': [],
        'certificate_eligible_emails': [],
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},
    },
    'lgdm2x': {
        'current_run':'UQx_LGDM2x_2T2017',
        'auditpassed_emails': [],
        'certificate_eligible_emails': [],       
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},
    },
    'lgdm3x': {
        'current_run':'UQx_LGDM3x_1T2018',
        'auditpassed_emails': [],
        'certificate_eligible_emails': [],       
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},       
    },
    'lgdm4x': {
        'current_run':'UQx_LGDM4x_1T2018',
        'auditpassed_emails': [],
        'certificate_eligible_emails': [],       
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},        
    },
}

def find_verified_less_than_4():
    table_name = mmCourse + '_coursecompletion'
    email_list = []
    query = "SELECT student_id, email FROM %s WHERE no_certificate_eligible < 4" % (table_name)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        email_list.append(record['email'])
    #print('less_than_4 no: ', len(email_list))
    return email_list


def find_auditpassed_next_course(course, record):
    #print('find_next_course', course, record)
    not_certificated = []
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    if course in not_certificated:
        not_certificated.remove(course)
    return not_certificated[0]

def find_certificate_eligible_next_course(record):
    not_certificated = []
    
    #print('record', record)
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    return not_certificated[0]
    
    
        
update_course_instancedetails_auditpassed()   
micromasters_courses_add_last_run()
less_than_4 = find_verified_less_than_4()

for course in pointed_course_instances: 
    print('course', course)
    #find_audit_passed(course)
    find_audit_passed(course, pointed_course_instances[course]['current_run'])
    find_certificate_eligible(course, pointed_course_instances[course]['current_run'])

    table_name = mmCourse + '_coursecompletion'  
    for audit_passed in pointed_course_instances[course]['auditpassed_emails']:
        print(audit_passed)
        audit_query = "SELECT * FROM %s WHERE no_certificate_eligible < 3 AND email = '%s'" % (table_name, audit_passed)
        audit_query_result = sqlite_connection.execute(audit_query)
        for audit_less in audit_query_result:
            next_course = find_auditpassed_next_course(course, audit_less)
            #print('next_course', next_course)
            
    for  certificate_eligible in pointed_course_instances[course]['certificate_eligible_emails']:
        print(certificate_eligible)
        verified_query = "SELECT * FROM %s WHERE no_certificate_eligible < 4 AND email = '%s'" % (table_name, certificate_eligible)
        verified_query_result = sqlite_connection.execute(verified_query)
        for verified_less in verified_query_result:
            next_course = find_certificate_eligible_next_course(verified_less)
            #print('next_course', next_course)
            

#pprint(pointed_course_instances)

print('The End.')

